In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load and prepare the datasets
data1 = pd.read_csv("../dataset/processed_data/Parkinson_CV_ensemble.tab", delimiter="\t")
data2 = pd.read_csv("../dataset/processed_data/Parkinson_FD_ensemble_pca.tab", delimiter="\t")

# Filter the last column as the label
label1 = data1.iloc[:, -1]
label2 = data2.iloc[:, -1]
label1 = label1.replace({'Co': 0, 'Pt': 1})
label2 = label2.replace({'Co': 0, 'Pt': 1})

# Remove the last column from the datasets
data1 = data1.iloc[:, :-1]
data2 = data2.iloc[:, :-1]

# Combine the datasets and labels
data = pd.concat([data1, data2], axis=1)
label = label1

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)

# Define the input shape
input_shape = (X_train.shape[1],)

# Define the attention mechanism layer
class Attention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

        attention_weights = tf.nn.softmax(self.V(score), axis=1)

        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

# Define the model architecture with attention mechanism
def create_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Dense(64, activation='relu')(inputs)
    hidden = tf.keras.layers.Dense(32, activation='relu')(x)
    context_vector, attention_weights = Attention(32)(x, hidden)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(context_vector)
    model = tf.keras.Model(inputs=inputs, outputs=output)
    return model

# Instantiate the model
model = create_model(input_shape)

# Define the optimizer and loss function
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.BinaryCrossentropy()

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=40, batch_size=8, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)

# Print the test accuracy
print('Test accuracy:', accuracy)



Epoch 1/40
27/27 [==============================] - 0s 5ms/step - loss: 181.1187 - accuracy: 0.6389 - val_loss: 71.3951 - val_accuracy: 0.6296
Epoch 2/40
27/27 [==============================] - 0s 1ms/step - loss: 135.7321 - accuracy: 0.5463 - val_loss: 55.3391 - val_accuracy: 0.6296
Epoch 3/40
27/27 [==============================] - 0s 1ms/step - loss: 116.4228 - accuracy: 0.5278 - val_loss: 51.0011 - val_accuracy: 0.7593
Epoch 4/40
27/27 [==============================] - 0s 1ms/step - loss: 99.5521 - accuracy: 0.6111 - val_loss: 52.0099 - val_accuracy: 0.7222
Epoch 5/40
27/27 [==============================] - 0s 2ms/step - loss: 76.6785 - accuracy: 0.6157 - val_loss: 37.6700 - val_accuracy: 0.7407
Epoch 6/40
27/27 [==============================] - 0s 1ms/step - loss: 75.9782 - accuracy: 0.5787 - val_loss: 38.7264 - val_accuracy: 0.7778
Epoch 7/40
27/27 [==============================] - 0s 1ms/step - loss: 76.1315 - accuracy: 0.5231 - val_loss: 37.2143 - val_accuracy: 0.7222
Epo

In [5]:
#side only
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load and prepare the datasets
data1 = pd.read_csv("../dataset/processed_data/Parkinson_CV_ensemble_sideonly.tab", delimiter="\t")
data2 = pd.read_csv("../dataset/processed_data/Parkinson_FD_ensemble_pca_sideonly.tab", delimiter="\t")

# Filter the last column as the label
label1 = data1.iloc[:, -1]
label2 = data2.iloc[:, -1]
label1 = label1.replace({'Co': 0, 'Pt': 1})
label2 = label2.replace({'Co': 0, 'Pt': 1})

# Remove the last column from the datasets
data1 = data1.iloc[:, :-1]
data2 = data2.iloc[:, :-1]

# Combine the datasets and labels
data = pd.concat([data1, data2], axis=1)
label = label1

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)

# Define the input shape
input_shape = (X_train.shape[1],)

# Define the attention mechanism layer
class Attention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

        attention_weights = tf.nn.softmax(self.V(score), axis=1)

        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

# Define the model architecture with attention mechanism
def create_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Dense(64, activation='relu')(inputs)
    hidden = tf.keras.layers.Dense(32, activation='relu')(x)
    context_vector, attention_weights = Attention(32)(x, hidden)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(context_vector)
    model = tf.keras.Model(inputs=inputs, outputs=output)
    return model

# Instantiate the model
model = create_model(input_shape)

# Define the optimizer and loss function
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.BinaryCrossentropy()

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=40, batch_size=8, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)

# Print the test accuracy
print('Test accuracy:', accuracy)



Epoch 1/40
27/27 [==============================] - 0s 5ms/step - loss: 143.9862 - accuracy: 0.5139 - val_loss: 82.7637 - val_accuracy: 0.5926
Epoch 2/40
27/27 [==============================] - 0s 1ms/step - loss: 90.7458 - accuracy: 0.5648 - val_loss: 67.6035 - val_accuracy: 0.5741
Epoch 3/40
27/27 [==============================] - 0s 968us/step - loss: 69.0181 - accuracy: 0.5556 - val_loss: 65.1301 - val_accuracy: 0.6111
Epoch 4/40
27/27 [==============================] - 0s 1ms/step - loss: 56.8428 - accuracy: 0.5417 - val_loss: 64.4019 - val_accuracy: 0.6111
Epoch 5/40
27/27 [==============================] - 0s 1ms/step - loss: 76.7579 - accuracy: 0.5417 - val_loss: 59.6829 - val_accuracy: 0.5741
Epoch 6/40
27/27 [==============================] - 0s 1ms/step - loss: 56.0320 - accuracy: 0.5694 - val_loss: 57.7542 - val_accuracy: 0.6296
Epoch 7/40
27/27 [==============================] - 0s 1ms/step - loss: 55.1356 - accuracy: 0.5602 - val_loss: 58.3745 - val_accuracy: 0.5185
Epo

In [3]:
pip install tensorflow


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
#exclude only
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load and prepare the datasets
data1 = pd.read_csv("../dataset/processed_data/Parkinson_CV_ensemble_excludediag.tab", delimiter="\t")
data2 = pd.read_csv("../dataset/processed_data/Parkinson_FD_ensemble_pca_exclude_diag.tab", delimiter="\t")

# Filter the last column as the label
label1 = data1.iloc[:, -1]
label2 = data2.iloc[:, -1]
label1 = label1.replace({'Co': 0, 'Pt': 1})
label2 = label2.replace({'Co': 0, 'Pt': 1})

# Remove the last column from the datasets
data1 = data1.iloc[:, :-1]
data2 = data2.iloc[:, :-1]

# Combine the datasets and labels
data = pd.concat([data1, data2], axis=1)
label = label1

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)

# Define the input shape
input_shape = (X_train.shape[1],)

# Define the attention mechanism layer
class Attention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

        attention_weights = tf.nn.softmax(self.V(score), axis=1)

        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

# Define the model architecture with attention mechanism
def create_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Dense(64, activation='relu')(inputs)
    hidden = tf.keras.layers.Dense(32, activation='relu')(x)
    context_vector, attention_weights = Attention(32)(x, hidden)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(context_vector)
    model = tf.keras.Model(inputs=inputs, outputs=output)
    return model

# Instantiate the model
model = create_model(input_shape)

# Define the optimizer and loss function
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.BinaryCrossentropy()

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=40, batch_size=8, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)

# Print the test accuracy
print('Test accuracy:', accuracy)



Epoch 1/40
27/27 [==============================] - 0s 4ms/step - loss: 94.9657 - accuracy: 0.5833 - val_loss: 56.6671 - val_accuracy: 0.6296
Epoch 2/40
27/27 [==============================] - 0s 1ms/step - loss: 100.5313 - accuracy: 0.5880 - val_loss: 38.8681 - val_accuracy: 0.7222
Epoch 3/40
27/27 [==============================] - 0s 1ms/step - loss: 103.7728 - accuracy: 0.5324 - val_loss: 36.4857 - val_accuracy: 0.6667
Epoch 4/40
27/27 [==============================] - 0s 1ms/step - loss: 90.8195 - accuracy: 0.5370 - val_loss: 45.1882 - val_accuracy: 0.6852
Epoch 5/40
27/27 [==============================] - 0s 1ms/step - loss: 68.2692 - accuracy: 0.5556 - val_loss: 44.9211 - val_accuracy: 0.6852
Epoch 6/40
27/27 [==============================] - 0s 1ms/step - loss: 69.4472 - accuracy: 0.5880 - val_loss: 29.1608 - val_accuracy: 0.7778
Epoch 7/40
27/27 [==============================] - 0s 1ms/step - loss: 73.4816 - accuracy: 0.5833 - val_loss: 41.4414 - val_accuracy: 0.7037
Epoc